In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import os

import pandas as pd
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v9 import CNN_v9

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v9((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v9/weights/"

def get_weights_paths(prefix):
    for directory, subdirectories, files in os.walk(prefix):
        yield from (os.path.join(directory, file) for file in files)

model_names = get_weights_paths(prefix)
model_names = list(model_names)

In [7]:
models = [CNN_v9((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [8]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6934 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 646ms/step - loss: 0.6935 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 690ms/step - loss: 0.6935 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 674ms/step - loss: 0.6934 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 682ms/step - loss: 0.6935 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 863ms/step - loss: 0.6935 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 651ms/step - loss: 0.6934 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 661ms/step - loss: 0.6934 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 634ms/step - loss: 0.6935 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 860ms/step - loss: 0.6934 - accuracy: 1.0000
Epoch 1/1
1/1 [==================

In [9]:
def as_rows(results):
    for name, cut_off_point_to_accuracy_with_difference in results.items():
        yield from ((f"{name}_{cut_off_point}", accuracy, counts - count_test) for cut_off_point, (accuracy, counts) in cut_off_point_to_accuracy_with_difference.items())

rows = as_rows(results)
rows = list(rows)

In [10]:
def calculate_mae_and_mse(rows):
    for *attributes, difference in rows:
        difference_mae = mae(difference)
        difference_mse = mse(difference)
        
        yield (*attributes, difference_mae, difference_mse)

mae_and_mse = calculate_mae_and_mse(rows)
mae_and_mse = list(mae_and_mse)

In [11]:
frame = pd.DataFrame(mae_and_mse, columns=["model", "accuracy", "mae", "mse"])
frame

,model,accuracy,mae,mse
0,CNN/CNN_v9/weights/cnn_v9_1e_6_5_0.1,0.000000,350.490000,122897.610000
1,CNN/CNN_v9/weights/cnn_v9_1e_6_5_0.2,0.000000,214.020000,46313.413333
2,CNN/CNN_v9/weights/cnn_v9_1e_6_5_0.30000000000...,0.000000,117.963333,14267.990000
3,CNN/CNN_v9/weights/cnn_v9_1e_6_5_0.4,0.000000,66.546667,4643.873333
4,CNN/CNN_v9/weights/cnn_v9_1e_6_5_0.5,9.680595,31.166667,1056.466667
...,...,...,...,...
850,CNN/CNN_v9/weights/cnn_v9_1e_6_99_0.5,81.353100,5.936667,48.716667
851,CNN/CNN_v9/weights/cnn_v9_1e_6_99_0.6000000000...,72.755814,8.583333,88.610000
852,CNN/CNN_v9/weights/cnn_v9_1e_6_99_0.7000000000...,63.612246,11.390000,146.636667
853,CNN/CNN_v9/weights/cnn_v9_1e_6_99_0.8,54.071877,14.363333,228.050000


In [12]:
frame.loc[frame.mae == frame.mae.min()]

,model,accuracy,mae,mse
120,CNN/CNN_v9/weights/cnn_v9_1e_6_18_0.4,90.548795,2.913333,13.866667


In [13]:
frame.loc[frame.mse == frame.mse.min()] --

,model,accuracy,mae,mse
75,CNN/CNN_v9/weights/cnn_v9_1e_6_13_0.4,90.369357,2.923333,13.363333


In [14]:
frame.loc[frame.accuracy == frame.accuracy.max()]

,model,accuracy,mae,mse
120,CNN/CNN_v9/weights/cnn_v9_1e_6_18_0.4,90.548795,2.913333,13.866667
